In [6]:
import cv2
import numpy as np
import csv
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
import keras
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import Convolution2D
from keras.layers import pooling
from keras.optimizers import Adam


_CORRECTION_NUM = 0.02

# ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

def helper(parent_dir):
    lines = []
    csv_file = './{}/driving_log.csv'.format(parent_dir)
    image_file_base = './{}/IMG/'.format(parent_dir)

    with open(csv_file) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)

    return_images = []
    return_measurements = []

    lines = iter(lines)
    # remove headers
    _ = next(lines)

    for line in lines:
        # center
        source_path = line[0]
        filename = source_path.split('/')[-1]
        current_path = image_file_base + filename
        image = cv2.imread(current_path)
        return_images.append(image)
        return_images.append(cv2.flip(image, 1))
        measurement = float(line[3])
        return_measurements.append(measurement)
        return_measurements.append(measurement * -1.0)

    return return_images, return_measurements


def train():
    parent_dirs = [
        'data',
        '1_recovery_data',
        'reverse-data',

        # NOTE: trying not using more-curves
        # it just bump into the bridge but it succesfully avoiding drifting into water
        # using just more-curves2 and more-curves3
    
        # 'more-curves', 
        'more-curves2-good-data',
        'more-curves3-good-data',
        'more-curves4',

    ]

    all_images = []
    all_measurements = []
    for the_dir in parent_dirs:
        _images, _measurements = helper(the_dir)
        print('dir: {}, images: {}'.format(the_dir, len(_images)))
        all_images.extend(_images)
        all_measurements.extend(_measurements)


    print('total images', len(all_images))
    print('total measurements', len(all_measurements))


    X_train = np.array(all_images)
    y_train = np.array(all_measurements)

    print(X_train.shape)
    print(y_train.shape)

    _DROPOUT_RATE = 0.2

    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 -0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((70,25), (0,0))))
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    model.add(Dropout(_DROPOUT_RATE))

    model.compile(optimizer=Adam(lr=0.001), loss='mse' , metrics=['accuracy'])
    print('Printing...')
    model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)
    # increasing epoch doesn't seem to help
    # model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=10)
    model.save('model.h5')
    print('Done')

train()

dir: data, images: 16072
dir: 1_recovery_data, images: 1914
dir: reverse-data, images: 1956
dir: more-curves2-good-data, images: 940
dir: more-curves3-good-data, images: 1452
total images 22334
total measurements 22334
(22334, 160, 320, 3)
(22334,)


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:88: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:89: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:90: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:91: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:92: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="

Printing...


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:102: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 17867 samples, validate on 4467 samples
Epoch 1/3
17867/17867 [==============================] - 23s 1ms/step - loss: 0.0137 - acc: 0.5604 - val_loss: 0.0264 - val_acc: 0.5391
Epoch 2/3
17867/17867 [==============================] - 22s 1ms/step - loss: 0.0125 - acc: 0.5604 - val_loss: 0.0270 - val_acc: 0.5391
Epoch 3/3
17867/17867 [==============================] - 22s 1ms/step - loss: 0.0125 - acc: 0.5604 - val_loss: 0.0256 - val_acc: 0.5391
Done
